In [1]:
# Ячейка для мгновенного обновления модулей
%load_ext autoreload
%autoreload 2

In [1]:
# libraries
import random
import pandas as pd
from itertools import permutations

from Bio import SeqIO

from word_to_biocode import WordToBiocode

import warnings 
warnings.simplefilter('ignore')

/home/ksu_marshmallow/miniconda3/envs/practical_bf/lib/python3.12/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


## Задача 🧬
Перевести слово (например, ФИО) в последовательность нуклеотидов (A, T, G, C)

## Решение

1. **Преобразование в двоичную систему**
    - В русском алфавите 33 буквы. Для их кодирования требуется **6 бит** (так как $2^6=64$, что достаточно для представления всех букв алфавита).
    - Каждую букву слова преобразуем в двоичный код длиной 6 цифр.

2. **Преобразование в четверичную систему**
    - Далее, этот двоичный код переводим в четверичную систему, где каждая цифра может быть 0, 1, 2 или 3.
    - Для этого каждые 2 бита двоичного кода превращаем в одну цифру четверичной системы.
    - Таким образом, *каждая буква будет представлена тремя цифрами четверичной системы*.

3. **Преобразование в нуклеотидную последовательность**
    - Каждая цифра четверичной системы (0, 1, 2, 3) кодируется нуклеотидом (например, 0 — A, 1 — C, 2 — G, 3 — T)
    - Таким образом, каждая буква слова будет закодирована в виде последовательности из 3 нуклеотидов (триплета).
    - **Длина закодированного слова** в нуклеотидах будет равна $3 \times \text{длина слова}$.

4. **Преобразование в аминокислотную последовательность**
    - Поскольку длина закодированной последовательности кратна трем, ее можно перевести в последовательность аминокислот (белок)


## Вероятность встретить закодированное слово в геноме

### Частота появления конкретной последовательности
Для случайного распределения нуклеотидов вероятность встретить определенную последовательность длиной $l$:
$$\frac{1}{4^l} = 4^{-l},$$ 

так как у каждого нуклеотида есть 4 возможных варианта (A, C, G, T)

Геном человека состоит из примерно $3.2 \cdot 10^9$ нуклеотидов. Т.е. возможных позиций для начала последовательности длиной $l$:
$3.2 \cdot 10^9 - l + 1 \approx 3.2 \cdot 10^9 \text{ н.}$

Чтобы вычислить, как часто можно встретить нашу закодированную последовательность длиной $l$ в геноме, умножаем количество возможных позиций на вероятность вообще получить такую последовательность. Тогда последовательность длиной $l$ будет встречаться в человеческом геноме со следующей частотой:
$$\frac{3.2 \cdot 10^9}{4^l} \text{ н.}$$

### Вероятность встретить последовательность хотя бы 1 раз
Вероятность встретить хотя бы одно вхождение последовательности в геноме можно выразить так: 
$$P = 1 - (1 - \frac{1}{4^l})^{3.2 \cdot 10^9}$$

## Реализация - перевод ФИО в ДНК и белок

In [3]:
sequences = []
protein_sequences = []
convertation = []

word = 'MALKOVAKE'

for nucl in list(permutations(['A', 'T', 'G', 'C'])):
    wordfinder = WordToBiocode(lang='eng', nucl=nucl)

    seq_word = wordfinder.encode(word, type_seq='DNA')
    protein_word = wordfinder.encode(word, type_seq='PROTEIN')

    sequences.append(seq_word)
    protein_sequences.append(protein_word)
    convertation.append(''.join(nucl))

    print(wordfinder.nucl_to_fourth)
    print(f'DNA: {seq_word}\nPROTEIN: {protein_word}')
    print('='*60)

{'A': '0', 'T': '1', 'G': '2', 'C': '3'}
DNA: ACAAAAAGCAGGACGTTTAAAAGGATA
PROTEIN: TKSRTFKRI
{'A': '0', 'T': '1', 'C': '2', 'G': '3'}
DNA: AGAAAAACGACCAGCTTTAAAACCATA
PROTEIN: RKTTSFKTI
{'A': '0', 'G': '1', 'T': '2', 'C': '3'}
DNA: ACAAAAATCATTACTGGGAAAATTAGA
PROTEIN: TKIITGKIR
{'A': '0', 'G': '1', 'C': '2', 'T': '3'}
DNA: ATAAAAACTACCATCGGGAAAACCAGA
PROTEIN: IKTTIGKTR
{'A': '0', 'C': '1', 'T': '2', 'G': '3'}
DNA: AGAAAAATGATTAGTCCCAAAATTACA
PROTEIN: RKMISPKIT
{'A': '0', 'C': '1', 'G': '2', 'T': '3'}
DNA: ATAAAAAGTAGGATGCCCAAAAGGACA
PROTEIN: IKSRMPKRT
{'T': '0', 'A': '1', 'G': '2', 'C': '3'}
DNA: TCTTTTTGCTGGTCGAAATTTTGGTAT
PROTEIN: SFCWSKFWY
{'T': '0', 'A': '1', 'C': '2', 'G': '3'}
DNA: TGTTTTTCGTCCTGCAAATTTTCCTAT
PROTEIN: CFSSCKFSY
{'T': '0', 'G': '1', 'A': '2', 'C': '3'}
DNA: TCTTTTTACTAATCAGGGTTTTAATGT
PROTEIN: SFY*SGF*C
{'T': '0', 'G': '1', 'C': '2', 'A': '3'}
DNA: TATTTTTCATCCTACGGGTTTTCCTGT
PROTEIN: YFSSYGFSC
{'T': '0', 'C': '1', 'A': '2', 'G': '3'}
DNA: TGTTTTTAGTAATGACCCTTTTAA

In [4]:
data = pd.DataFrame({"encoding": convertation, 
              "DNA_seq": sequences, 
              "PROTEIN_seq": protein_sequences})

data.sample(5, random_state=17)

encoding                      DNA_seq PROTEIN_seq
10     TCAG  TGTTTTTAGTAATGACCCTTTTAATCT   CF***PF*S
0      ATGC  ACAAAAAGCAGGACGTTTAAAAGGATA   TKSRTFKRI
21     CTGA  CACCCCCGACGGCAGTTTCCCCGGCTC   HPRRQFPRL
11     TCGA  TATTTTTGATGGTAGCCCTTTTGGTCT   YF*W*PFWS
5      ACGT  ATAAAAAGTAGGATGCCCAAAAGGACA   IKSRMPKRT

## Переводим ген в слова
- В нашем алфавите нет пунктуации и пробелов
- Можно добавить их в наш словарь. Было бы логично сделать стоп-кодоны точкой, а самый часто-встречающийся кодон - пробелом.
- Можно обучить модель (н-р RNN) на парах (без пунктуации, с пунктуацией), чтобы она предсказывала, где вставить пробелы, запятые, точки и другие символы. Но смысла нет, так как декодирование генов в приницпе бессмысленно)

In [5]:
word = 'hi'
seq_to_word = WordToBiocode(lang='eng')

assert word.lower() == seq_to_word.decode(seq_to_word.encode(word))

In [6]:
seq_to_word.encode(word)

'ATCAGA'

### Читаем .fasta и декодируем различные гены

In [7]:
def read_fasta(path_to_fasta: str):
    sequence = []
    for record in SeqIO.parse(path_to_fasta, "fasta"):
        sequence.append(str(record.seq))
    sequence = ''.join(sequence)
    return sequence

#### APOE

In [8]:
apoe_seq = read_fasta('genes/APOE.fasta')
apoe_word = seq_to_word.decode(apoe_seq)
apoe_word[:100]

'llcjksaksltwikplipscllknnhfrnzzhenmsinuaybsebmunkwigykbnngkfypfxreiasgazfkwegwqkkjwfjxxibkfkrhppivhe'

#### HBB

In [9]:
hbb_seq = read_fasta('genes/HBB.fasta')
hbb_word = seq_to_word.decode(hbb_seq)
hbb_word[:100]

'mvmnxnldamhykijuzkyogcwkrtcmukienmcnvenwrvpktpkxvvngdcccavjdcpvmjzcdxkjyzvvujzkcejdkvigdigfcvjxvrwxe'

#### BRCA1

In [10]:
brca_word = seq_to_word.decode(read_fasta('genes/BRCA1.fasta'))
brca_word[:100]

'nokxdkkadicafrxwcctagugiasjjykdjpmrvbvydxpiicjrigeqatcaudwwclryavxlwmjgmefwakaendhqcgcvampaxtidahkad'

#### TP53

In [11]:
tp53_word = seq_to_word.decode(read_fasta('genes/TP53.fasta'))
tp53_word[:100]

'ajkvxvyisdfxignnyalbyvrymgyikiylbiplltpplhxzxdvxlzwwnzyxpmzyisrjkavzvyavmszyyytqxniqnoyzwiiikkqlnapn'

#### FTO

In [12]:
fto_word = seq_to_word.decode(read_fasta('genes/FTO.fasta'))
fto_word[:100]

'lzpfqhhvnkbliinmkkhtvjlgcpnczilcpkvgakkwkeugufwwabcpxxqfzztfiyytvegyiybhzlzwwydjwzueyhvdtftyzjgyqqiv'

плохая шутка - даже ген напоминает о больном:

In [13]:
'fat' in fto_word

True

### Поиск слов в декодированном гене

In [14]:
import re
import nltk
from nltk.corpus import words

# nltk.download('words')

def find_words(sequence, word_list, min_length=1, max_length=float('inf'), ignore_punctuation=True):
    found_words = []
    
    # пунктуации сейчас все равно нет, но мало ли добавлю
    if ignore_punctuation:
        sequence = re.sub(r'[^\w\s]', '', sequence)

    for word in word_list:
        if min_length <= len(word) <= max_length:
            if re.search(word, sequence):
                found_words.append(word)

    return found_words

In [15]:
word_list = set(words.words())
print(len(word_list))

235892


In [16]:
%%time
apoe_found = find_words(apoe_word, word_list, min_length=3)
apoe_found[:20]

CPU times: user 3.48 s, sys: 429 μs, total: 3.48 s
Wall time: 3.81 s


['kil',
 'lin',
 'hei',
 'fig',
 'mun',
 'six',
 'kip',
 'hen',
 'yoi',
 'wig',
 'wun',
 'vau',
 'inn',
 'sin',
 'lip',
 'gaz',
 'pub',
 'tab',
 'linn',
 'poky']

In [17]:
%%time
tp53_found = find_words(tp53_word, word_list, min_length=3)
tp53_found[:20]

CPU times: user 3.56 s, sys: 458 μs, total: 3.56 s
Wall time: 3.9 s


['pua',
 'yis',
 'kat',
 'yalb',
 'beg',
 'urf',
 'noy',
 'ani',
 'thy',
 'nap',
 'alb',
 'aka']

In [18]:
%%time
hbb_found = find_words(hbb_word, word_list, min_length=3)
hbb_found[:20]

CPU times: user 3.58 s, sys: 2.59 ms, total: 3.58 s
Wall time: 3.87 s


['dam', 'ary', 'dak', 'wem', 'dig', 'nix', 'arx', 'baa', 'ewe', 'aka']

In [19]:
%%time
brca_found = find_words(brca_word, word_list, min_length=3)
print(len(brca_found))

98
CPU times: user 3.54 s, sys: 1.43 ms, total: 3.54 s
Wall time: 3.82 s


In [20]:
sorted(brca_found, key=len, reverse=True)[:20]

['pali',
 'bead',
 'vamp',
 'gleg',
 'wice',
 'lied',
 'puka',
 'waka',
 'clue',
 'masa',
 'cusk',
 'lue',
 'ade',
 'shy',
 'pig',
 'wap',
 'naa',
 'bal',
 'elt',
 'edh']

In [21]:
%%time
fto_found = find_words(fto_word, word_list, min_length=3)
print(len(fto_found))

1476
CPU times: user 12.5 s, sys: 1.97 ms, total: 12.5 s
Wall time: 11.1 s


FTO просто мем:

In [22]:
sorted(fto_found, key=len, reverse=True)[:20]

['urushi',
 'laical',
 'revere',
 'navel',
 'gaily',
 'arear',
 'rever',
 'acara',
 'stiff',
 'diver',
 'crash',
 'fusee',
 'vagas',
 'guaba',
 'abear',
 'remex',
 'dwalm',
 'pasty',
 'frier',
 'idiot']

## Укладываем белок
- Средняя длина белка - 500-600 а.к.
- Нужно сгенерировать текст длиной 500-600 символов без пунктуации. Я просто возьму отрывок из Алисы в стране чудес
- Дальше переведем текст в белок и попытаемся его уложить альфа-фолдом

In [23]:
import string

In [35]:
text = 'Волшебный мир, который открывается перед Алисой, наполнен странными существами и невероятными приключениями. Она встречает Чеширского кота, который исчезает, оставляя лишь улыбку, и слышит его мудрые слова о том, что, если идти в любом направлении, всегда можно оказаться где-то. Каждое новое столкновение с героями этого мира — будь то Безумный Шляпник или Соня — бросает вызов ее восприятию реальности и логики, заставляя задуматься о том, что значит быть собой в таком странном месте, где правила меняются по щелчку пальцев и где каждый день может превратиться в захватывающее приключение или же в полное абсурда событие. Алиса осознает, что в этом удивительном месте нет четких границ между сном и реальностью, и только она сама может решить, что действительно имеет значение для нее. В этот волшебный мир она попадает, чтобы учиться, расти и открывать для себя что-то новое, даже если это что-то вызывает смятение и удивление. В то время как сама Алиса продолжает исследовать этот бесконечно загадочный мир, читатели не могут не задуматься о том, как много чудесного и необычного вокруг них, если только они откроют свои сердца и умы, чтобы воспринять это с воодушевлением.'

punctuation = string.punctuation + '-' + '—'
regex_pattern = f"[{re.escape(punctuation)}]"
cleaned_text = re.sub(regex_pattern, '', text.replace(' ', ''))

In [36]:
print(len(cleaned_text))
cleaned_text[:100]

966


'ВолшебныймиркоторыйоткрываетсяпередАлисойнаполненстраннымисуществамииневероятнымиприключениямиОнавст'

In [43]:
text2seq = WordToBiocode(lang='rus')

protein_text = text2seq.encode(cleaned_text, type_seq='PROTEIN').replace('*', '')
print(len(protein_text))
protein_text

874


'KTTCINTSRTSYSTYTYSRTYSYSKKIYEIYIIKTSTRTKTTTITYYKTTSTSLWIYKKTSSTIKIYTEYTSTSYSSTSITSETSTTKKYYIKIYICSYSTNTSTYKSTYTYSRSIRKIYTYKKTEETSCSLTSNSLSTSCSYINTTLIYSITTKKTYTTYTITSSIYSKTSNTTTKYKKTITSSKINIKTTITTTSKRKYSENIIYTSKIITITTKTIYTTSTTKITSINIYTETSSYTNTTSYKNLISYTNIRLTTSRCTETSSSTSTTENYTKIYKSRTKIIKTYSEYSSYIKTSTTYSSTTNSSSRKYKKTEERKILTKYSETYTTYTRTKSYNSYSTNTRKYKSTTYYKTTTTTIYINIIYKKSTKTITESYETWITSLKTSFIKSNIISKIISRIITSTTIIYYIKYKYSYSEKRKLKKYSKKSWIIYSSTSITSISTSIIKTTTTIKNLYIKTNSYSIKTSKTTRTKIYYTKSYTTLISKSYITSTTTTIYITIYIYSSLNYKTSFTIIILTTTSYIKTSTTYSSSYTTSSTTTKKTKTTIIYYICSYSYTIIRYKSYITSTTSTIIYRTKITSIITETIIKSYTYKTTCINTSRTSYTTKTKIKIYYTNSLSYSEYKYSSTYSYSKKYSITEINEYTYTTTKTIIKIIITSSYTYTYTKSRSKKIYTEYITSISLISKTITSIKYTKYITESKSKTKKTSKYTITTIKIYSTIITKKYSSYTYNISTTITTRKNKITTSRTSYSYKYITSTITTNLYTIRKILTKYSETYTTSKSTTTNTLIITTNTSTITNSTTNTKTSYLNTSLITSYTTSSTTTSTYSYTSYKTSIYIFKSLTSYTNSKTYSTEYSSYTKTTILCIKTITSIT'

Эту последовательность я отправляю в [AlphaFold](https://colab.research.google.com/github/deepmind/alphafold/blob/main/notebooks/AlphaFold.ipynb)